In [1]:
import json
from dotenv import load_dotenv
from functools import reduce
import numpy as np
import operator
import os
import pandas as pd
import pymongo
from pymongo import MongoClient
import requests

In [2]:
load_dotenv()

True

In [12]:
paris_ = {'type': 'Point', 'coordinates': [48.87241163769417,2.3317031342524017]}

In [13]:
melbourne_cbd = {'type': 'Point', 'coordinates': [-37.81227175879301,144.96262186884945]}

In [37]:
sao_paulo_zc = {'type': 'Point', 'coordinates': [-23.533215960575276,-46.639408758375076]}

#### GEOQUERIES

In [14]:
client = MongoClient("localhost:27017")
db = client.get_database("GeoSpatialProyect")
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'GeoSpatialProyect')

In [15]:
transport = db.get_collection("transport")

In [16]:
gastronomy = db.get_collection("gastronomy")

In [17]:
schools = db.get_collection("schools")

In [18]:
veterinary = db.get_collection("veterinary")

In [19]:
basketball = db.get_collection("basketball")

In [20]:
transport.find_one().keys()

dict_keys(['_id', 'name', 'categories', 'latitude', 'longitude', 'location'])

#### TRANSPORT QUERIES - PARIS

In [21]:
cond1 = {'categories.name': 'Airport'}
cond2 = {"location": {"$near": {"$geometry": paris_, "$minDistance": 0 ,"$maxDistance": 20000}}}
gquery_p1 = {'$and': [cond1, cond2]}

In [33]:
cond1 = {'categories.name': 'Train Station'}
cond2 = {"location": {"$near": {"$geometry": paris_, "$minDistance": 0 ,"$maxDistance": 2000}}}
gquery_p2 = {'$and': [cond1, cond2]}

In [42]:
cond1 = {'categories.name': 'Bus Stop'}
cond2 = {"location": {"$near": {"$geometry": paris_, "$minDistance": 0 ,"$maxDistance": 2000}}}
gquery_p3 = {'$and': [cond1, cond2]}

In [55]:
cond1 = {'categories.name': 'Transportation Service'}
cond2 = {"location": {"$near": {"$geometry": paris_, "$minDistance": 0 ,"$maxDistance": 2000}}}
gquery_p4 = {'$and': [cond1, cond2]}

In [44]:
len(list(transport.find(gquery_p1)))

2

In [45]:
len(list(transport.find(gquery_p2)))

1

In [46]:
len(list(transport.find(gquery_p3)))

3

In [58]:
len(list(transport.find(gquery_p4)))

0

In [7]:
score_p = {'Conditions': ['Airports', 'Commuter Station', 'Airport Shuttle', 'Schools', 'Vegan Restaurants', 'Veterinaries', 'Starbucks Cafe', 'Basketball Court'],
           'Geoqueries': [2.0, 3.0, 4.0, 8.0, 4.0, 2.0, 1.0, 3.], 'Weights': [0.19, 0.18, 0.17, 0.14, 0.12, 0.1, 0.05, 0.05]}

In [8]:
score_p = pd.DataFrame(score_p)

In [9]:
score_p

,Conditions,Geoqueries,Weights
0,Airports,2.0,0.19
1,Commuter Station,3.0,0.18
2,Airport Shuttle,4.0,0.17
3,Schools,8.0,0.14
4,Vegan Restaurants,4.0,0.12
5,Veterinaries,2.0,0.10
6,Starbucks Cafe,1.0,0.05
7,Basketball Court,3.0,0.05


In [283]:
w = np.array([0.19, 0.18, 0.17, 0.14, 0.12, 0.1, 0.05, 0.05])

In [ ]:
g = np.array([2, 3, 4, 8, 4, 2, 1, 3])

In [ ]:
s = w * g

In [ ]:
score_p['Score'] = s